In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# EDA 
Files

    train.csv - the training set
    test.csv - the test set
    sample_submission.csv - a sample submission file in the correct format

Columns

    id - unique ID for excerpt
    url_legal - URL of source - this is blank in the test set.
    license - license of source material - this is blank in the test set.
    excerpt - text to predict reading ease of
    target - reading ease
    standard_error - measure of spread of scores among multiple raters for each excerpt. Not included for test data.

##  Import packages 

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

# Import data :

In [ ]:
# Import the datasets
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

# EDA train data 

In [ ]:
# Perform EDA on the train
train.shape ,train.dtypes

In [ ]:
train.head()

In [ ]:
train.excerpt[0]

In [ ]:
train.describe(include='all')

# Initial thoughts

    Extracting information from the "Excerpt" column will be key to this analysis

            All of the values are unique
            EDA required for text will help with future discoveries

    Target column shows a broad range of values

            Distribution shows a larger proportion of negative values (perform a Histogram and box plot to confirm)
            There appears to be a negative skew with the mean value lower than the median (50th percentile)



# EDA test data 

In [ ]:
# Perform EDA on the test
test.head()

In [ ]:
test.shape

In [ ]:
test.describe(include='all')

Initial thoughts of the test dataset

    Excerpt is the key variable
    Seven values mean that the validation of the training set will be key to optimise the model. Developing a hold out sample on this size could help
# Data Visualizations

In [ ]:
# The histogram provides details on the distribution of the variable. Including the box plot shows key parameter summary values.
# By using plotly we are able to hover over the values and easily understand how the values compare
fig = px.histogram(train, x="target",
                   marginal="box")
fig.show()



2. Data discovery

Key challenge is to understand the difficulty of the readability challenge. When reviewing how difficult a text is there are a few key areas of interest:

    - Word difficulty

            Vocabulary lists : can be used to highlight the proportion of comman words used. The less common a word is the more difficult it can be perceived and understood to be
            Word length : longer words are usually seen as more difficult that short. Therefore a correlation could be constructed between the word length and text difficulty

    - Sentence difficulty

            Sentence length : longer sentences lead to more difficult text. Have to be aware that the inclusion of colon and semi-colon can impact sentence length as well as the full stop



In [ ]:
# Extract insights from the excerpt variable
import spacy
# Initialise spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Perform initial test on one excerpt
sample1 = train.loc[0, 'excerpt']
sample1

In [ ]:
# Create the spacy doc item for review
doc = nlp(sample1)
doc

In [ ]:
# Reviewing the token, lemma and stopword for each token (item)
print(f"Token \t\tLemma \t\tStopword".format('Token', 'Lemma', 'Stopword'))
print("-"*40)
# Review the first 10  values to test the output
for token in doc[:10]:
    print(f"{str(token)}\t\t{token.lemma_}\t\t{token.is_stop}\t\t{len(token)}")

# Stop words :

In [ ]:
# A few different options for stopwords, spacy and nltk. Lets compare
import nltk
from nltk.corpus import stopwords
# Compare the differences
nltk_set = set(stopwords.words('english'))
spacy_set = set(nlp.Defaults.stop_words)

# Union - all values
union = nltk_set.union(spacy_set)
# Intersection - seen in both sets
inter = nltk_set.intersection(spacy_set)
print(f"Seen in both : {len(inter)} \n {inter}")
# Remainder - differences between sets
nltk_extra = nltk_set - inter
spacy_extra = spacy_set - inter
print(f"Extra NLTK : {len(nltk_extra)} \n {nltk_extra}")
print(f"Extra Spacy : {len(spacy_extra)} \n {spacy_extra}")

Spacy appears to cover a wider range of stopwords. Adding the additional 56 words from the NLTK could help to increase the scope of stopwords available for use.

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer
#instantiate CountVectorizer() 
cv=CountVectorizer() 

# this steps generates word counts for the words in the sample doc
word_count_vector=cv.fit_transform(train.excerpt)

word_count_vector.shape

In [ ]:
# Compute the IDF values
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])
df_idf.describe()

In [ ]:
# Lets create a dictionary to review the key phrase outputs
from collections import defaultdict, Counter

# Returns integers that map to parts of speech
counts_dict = doc.count_by(spacy.attrs.IDS['POS'])

# Print the human readable part of speech tags
for pos, count in counts_dict.items():
    human_readable_tag = doc.vocab[pos].text
    print(human_readable_tag, count)

In [ ]:
pos_counts = defaultdict(Counter)
for token in doc:
    pos_counts[token.pos][token.orth] += 1
    
for pos_id, counts in sorted(pos_counts.items()):
    pos = doc.vocab.strings[pos_id]
    for orth_id, count in counts.most_common():
        print(pos, count, doc.vocab.strings[orth_id], len(doc.vocab.strings[orth_id]))

In [ ]:
# Expanding named entities
for entity in doc.ents:
    print(entity.text, entity.label_)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])
print("Number of sentences", len([*doc.sents]))
print("Sentiment", doc.sentiment)

# Understand the length of sentences
for sent in doc.sents:
    print(sent.start_char, sent.end_char, (sent.end_char - sent.start_char))

In [ ]:
from spacy import displacy

# Display the entities within a sentence
displacy.render(doc, style='ent', jupyter=True)

# 3. Create the datasets for training the models

In [ ]:
# Lets apply the nlp instance to each excerpt
train['excerpt_scy'] = train['excerpt'].apply(nlp)

In [ ]:
# Check the data type for the updated column
type(train.loc[0, 'excerpt_scy'])
train.head()

# Feature Extraction :


In [ ]:
# Create the class methods required to run the analysis
class NLPMethods():
    # Create constructor for the class
#     def __init__():
    
    # Number of sentences
    def number_sentences(self, nlp_text):
        return len([*nlp_text.sents])
    
    # Average length of sentence
    def average_sentence_length(self, nlp_text):
        sent_length = list()
        for sent in nlp_text.sents:
            sent_length.append(sent.end_char - sent.start_char)
        return np.mean(sent_length)
    
    # Part of speech tags
    def part_of_speech_tags(self, nlp_text):
        counts_dict = nlp_text.count_by(spacy.attrs.IDS['POS'])
        counts_dict1 = {}
        # Extract the text that matches to the POS value
        for k, v in counts_dict.items():
            counts_dict1[nlp_text.vocab[k].text] = v
        return counts_dict1
    
    # Number of spaces
    def number_spaces(self, nlp_text):
        dict_pos = self.part_of_speech_tags(nlp_text)
        if dict_pos.get('SPACE') != None:
            space = dict_pos.get('SPACE')
        else:
            space = 0
        return space
    
    # Part of speech tags - including the word counts
    def word_counts(self, nlp_text):
        pos_counts = defaultdict(Counter)
        for token in nlp_text:
            pos_counts[token.pos][token.orth] += 1
        
        # Create dictionary for the word counts
        word_counts_dict = {}
        for pos_id, counts in sorted(pos_counts.items()):
            pos = nlp_text.vocab.strings[pos_id]
            for orth_id, count in counts.most_common():
                word_counts_dict[nlp_text.vocab.strings[orth_id]] = {'count':count, 
                                                                     'length':len(nlp_text.vocab.strings[orth_id]), 
                                                                     'pos':pos}
        return word_counts_dict
    
    # Number of words
    def number_words(self, nlp_text):
        dict_word_counts = self.word_counts(nlp_text)
        return len(dict_word_counts.items())
    
    # Longest word
    def longest_word(self, nlp_text):
        dict_word_counts = self.word_counts(nlp_text)
        df = pd.DataFrame(dict_word_counts).T.reset_index().rename(columns={'index':'variable'})
        return max(df['length'])

In [ ]:
# Add columns for the spacy doc
train['num_sentences'] = train['excerpt_scy'].apply(NLPMethods().number_sentences)
train['avg_sentence_length'] = train['excerpt_scy'].apply(NLPMethods().average_sentence_length)
train['pos_dict'] = train['excerpt_scy'].apply(NLPMethods().part_of_speech_tags)
train['num_space'] = train['excerpt_scy'].apply(NLPMethods().number_spaces)
train['wc_dict'] = train['excerpt_scy'].apply(NLPMethods().word_counts)
train['num_words'] = train['excerpt_scy'].apply(NLPMethods().number_words)
train['longest_word'] = train['excerpt_scy'].apply(NLPMethods().longest_word)

In [ ]:
train['avg_sentence_length'].mean()

In [ ]:
train.sample(5)

In [ ]:
# Review the max value target variable
max_val = np.max(train['target'])
train_max = train.loc[(train['target']==max_val), :]
train_max

In [ ]:


# Check the reason for the largest target value
type(train.loc[2829, 'excerpt_scy'])
train.loc[2829, 'excerpt_scy']
train.loc[2829, 'excerpt']



It appears that the word "paleontologists" could be causing the difficulty?




It appears that the word "paleontologists" could be causing the difficulty?


In [ ]:
# Review the min value target variable
min_val = np.min(train['target'])
train_min = train.loc[(train['target']==min_val), :]
train_min

In [ ]:
# Check the reason for the smallest target value
type(train.loc[1705, 'excerpt_scy'])
train.loc[1705, 'excerpt_scy']


# EDA of new variables

In [ ]:


# Import libraries
import seaborn as sns



# Correlation analysis
df = train.loc[:, ['id', 'target', 'num_sentences', 'avg_sentence_length', 'num_space', 'num_words', 'longest_word']]
df.head()

cor = df.corr()
sns.heatmap(cor, annot=True)
plt.show()



In [ ]:
X = df.drop('id', axis=1)
X.dtypes

In [ ]:
# Review a scatter matrix
fig = px.scatter_matrix(X)
fig.show()